In [2]:
from argparse import Namespace
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

from vocabulary import Vocabulary

%matplotlib inline

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (14, 6)

START_TOKEN = "^"
END_TOKEN = "_"
IGNORE_INDEX_VALUE = -1

## Class Definitions 

Data Model:
- Raw data
- Vectorizer
- Vectorized Data
- Data generator

In [3]:
class RawTrumpTweets(object):
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        
    def get_data(self):
        return self.data  

# vectorizer

class TrumpTweetVectorizer(object):
    def __init__(self, word_vocab, max_seq_length):
        self.word_vocab = word_vocab
        self.max_seq_length = max_seq_length
        
    def save(self, filename):
        vec_dict = {"word_vocab": self.word_vocab.get_serializable_contents(),
                    'max_seq_length': self.max_seq_length}

        with open(filename, "w") as fp:
            json.dump(vec_dict, fp)
        
    @classmethod
    def load(cls, filename):
        with open(filename, "r") as fp:
            vec_dict = json.load(fp)

        vec_dict["word_vocab"] = Vocabulary.deserialize_from_contents(vec_dict["word_vocab"])
        return cls(**vec_dict)

    @classmethod
    def fit(cls, tweet_df):
        vocab = Vocabulary(use_unks=False,
                           use_start_end=True,
                           use_mask=True,
                           start_token=START_TOKEN,
                           end_token=END_TOKEN)
        max_seq_length = 0
        for text in tweet_df.text:
            split_text = text.split(" ")
            vocab.add_many(split_text)
            if len(split_text) > max_seq_length:
                max_seq_length = len(split_text)
        max_seq_length = max_seq_length + 2
        return cls(vocab, max_seq_length)

    @classmethod
    def fit_transform(cls, tweet_df, split='train'):
        vectorizer = cls.fit(tweet_df)
        return vectorizer, vectorizer.transform(tweet_df, split)

    def transform(self, tweet_df, split='train'):
        tweet_df = tweet_df[tweet_df.split==split].reset_index()
        num_data = len(tweet_df)
        
        x_words = np.zeros((num_data, self.max_seq_length), dtype=np.int64)
        y_words = np.ones((num_data, self.max_seq_length), dtype=np.int64) * IGNORE_INDEX_VALUE

        for index, row in tweet_df.iterrows():
            converted = list(self.word_vocab.map(row.text.split(' '), include_start_end=True))
            x_version = converted[:-1]
            y_version = converted[1:]
            
            x_words[index, :len(x_version)] = x_version
            y_words[index, :len(y_version)] = y_version
            
        return VectorizedTrumpTweets(x_words, y_words)

# vec data


class VectorizedTrumpTweets(Dataset):
    def __init__(self, x_words, y_words):
        self.x_words = x_words
        self.y_words = y_words

    def __len__(self):
        return len(self.x_words)

    def __getitem__(self, index):
        return {'x_words': self.x_words[index],
                'y_words': self.y_words[index],
                'x_lengths': len(self.x_words[index].nonzero()[0])}

# data generator

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, use_cuda=False):
        
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = Variable(tensor)
            if use_cuda:
                out_data_dict[name] = out_data_dict[name].cuda()
        yield out_data_dict

## Class definitions for the model

In [4]:
def new_parameter(*size):
    out = Parameter(torch.FloatTensor(*size))
    torch.nn.init.xavier_normal(out)
    return out

def column_gather(y_out, x_lengths):
    '''Get a specific vector from each batch datapoint in `y_out`.

    More precisely, iterate over batch row indices, get the vector that's at
    the position indicated by the corresponding value in `x_lengths` at the row
    index. 

    Args:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, sequence, feature)
        x_lengths (torch.LongTensor, torch.cuda.LongTensor)
            shape: (batch,)

    Returns:
        y_out (torch.FloatTensor, torch.cuda.FloatTensor)
            shape: (batch, feature)
    '''
    x_lengths = x_lengths.long().data.cpu().numpy() - 1
    # alternatively:
    # out = []
    # for batch_index, column_index in enumerate(x_lengths):
    #     out.append(y_out[batch_index, column_index])
    # return torch.stack(out)
    return torch.stack([y_out[batch_index, column_index]
                        for batch_index, column_index in enumerate(x_lengths)])

class ExplicitRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super(ExplicitRNN, self).__init__()
        self.W_in2hid = new_parameter(input_size, hidden_size)
        self.W_hid2hid = new_parameter(hidden_size, hidden_size)
            
        self.b_hid = new_parameter(1, hidden_size)
        
        self.hidden_size = hidden_size

        self.batch_first = batch_first
    
    def _compute_next_hidden(self, x, h):
        return F.tanh(x.matmul(self.W_in2hid) + 
                      h.matmul(self.W_hid2hid) + 
                      self.b_hid)

    def forward(self, x_in, hid_t=None):
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
            x_in = x_in.permute(1, 0, 2)
        else:
            seq_size, batch_size, feat_size = x_in.size()

        hiddens = []
        if hid_t is None:
            hid_t = Variable(torch.zeros((batch_size, self.hidden_size)))
        
        if x_in.is_cuda:
            hid_t = hid_t.cuda()
            
        for t in range(seq_size):
            x_t = x_in[t]
            hid_t = self._compute_next_hidden(x_t, hid_t)
            
            hiddens.append(hid_t)
        hiddens = torch.stack(hiddens)

        if self.batch_first:
            hiddens = hiddens.permute(1, 0, 2)

        return hiddens
    
    
class WordRNN(nn.Module):
    def __init__(self, embedding_size, in_vocab_size, out_vocab_size, hidden_size, 
                 batch_first=True):
        super(WordRNN, self).__init__()
        
        self.emb = nn.Embedding(embedding_dim=embedding_size, 
                                num_embeddings=in_vocab_size, 
                                padding_idx=0)
        self.fc = nn.Linear(in_features=hidden_size, out_features=out_vocab_size)
        
        self.rnn = ExplicitRNN(input_size=embedding_size, 
                               hidden_size=hidden_size, 
                               batch_first=batch_first)
    
    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        x_in = self.emb(x_in)
        y_out = self.rnn(x_in)

        dim0, dim1, dim2 = y_out.size()
        y_out = y_out.contiguous().view(-1, dim2)

        y_out = self.fc(y_out)

        # optionally apply the softmax
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        y_out = y_out.view(dim0, dim1, -1)
        
        return y_out
    
def normalize_sizes(net_output, y_true):
    net_output = net_output.cpu()
    y_true = y_true.cpu()
    if len(net_output.size()) == 3:
        net_output.contiguous()
        net_output = net_output.view(-1, net_output.size(2))
    if len(y_true.size()) == 2:
        y_true.contiguous()
        y_true = y_true.view(-1)
    return net_output, y_true

def sequence_loss(net_output, y_true, loss_func=F.cross_entropy):
    net_output, y_true = normalize_sizes(net_output, y_true)
    return F.cross_entropy(net_output, y_true, ignore_index=IGNORE_INDEX_VALUE)

def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.cpu().max(dim=1)
    
    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()
    
    n_correct = (correct_indices * valid_indices).sum().data[0]
    n_valid = valid_indices.sum().data[0]

    return n_correct / n_valid * 100

### sampling functions

in the last couple of examples, we've seen sampling.  we include the sampling code up front here so that we can sample during training to get a sense of what's going on!

In [5]:
def sample(emb, rnn, fc, h_t=None, idx_t=None, n=20, temp=1):
    hiddens = [h_t]
    indices = [idx_t]
    out_dists = []
    
    for t in range(n):
        x_t = emb(idx_t)
        h_t = rnn._compute_next_hidden(x_t, h_t)
        
        y_t = fc(h_t)
        y_t = F.softmax( y_t / temp, dim=1)
        idx_t = torch.multinomial(y_t, 1)[:, 0]
        
        
        hiddens.append(h_t)
        indices.append(idx_t)
        out_dists.append(y_t)
     
    indices = torch.stack(indices).squeeze().permute(1, 0)
    return indices

def make_initial_hidden(batch_size, hidden_size):
    out = Variable(torch.ones(batch_size, hidden_size))
    return out

def make_initial_x(batch_size, vectorizer):
    out = Variable(torch.ones(batch_size) * vectorizer.word_vocab.start_index).long()
    return out

def decode_one(vectorizer, seq):
    out = []
    for i in seq:
        if vectorizer.word_vocab.start_index == i:
            continue
        if vectorizer.word_vocab.end_index == i:
            return ' '.join(out)
        out.append(vectorizer.word_vocab.lookup(i))
    return ' '.join(out)
            
def decode_matrix(vectorizer, mat):
    mat = mat.cpu().data.numpy()
    return [decode_one(vectorizer, mat[i]) for i in range(len(mat))]

## Make, Train, and Eval

In [6]:
args = Namespace(
    trump_csv="data/trump.csv",
    glove_filename="data/glove.6B.100d.txt",
    batch_size=128,
    cuda=False,
    learning_rate=0.001,
    num_epochs=100,
    load_zoo_model=True,
    zoo={
        'filename': 'modelzoo/wordrnn_emb100_hid64_trump_tweets_predict.state',
        'vocab': 'modelzoo/trump_twitter.vocab',
        'comments': 'pre-trained trump sequence prediction (& generation)',
        'parameters': {
            'embedding_size': 100,
            'hidden_size': 64
        }
    }

)


args.cuda = args.cuda and torch.cuda.is_available()

if args.cuda:
    print("CUDA is enabled")
else:
    print("CUDA is not enabled")

CUDA is not enabled


In [5]:
# optional: set this to false to learn from scratch!
# args.load_zoo_model = False

In [10]:
raw_data = RawTrumpTweets(args.trump_csv).get_data()

if os.path.exists(args.zoo['vocab']):
    vectorizer = TrumpTweetVectorizer.load(args.zoo['vocab'])
    print("Loading vectorizer!")
else:
    vectorizer = TrumpTweetVectorizer.fit(raw_data)
    print("Creating a new vectorizer.")

train_dataset = vectorizer.transform(raw_data, split='train')
test_dataset = vectorizer.transform(raw_data, split='test')

zoo_params = args.zoo['parameters']

net = WordRNN(embedding_size=zoo_params['embedding_size'], 
              hidden_size=zoo_params['hidden_size'],
              in_vocab_size=len(vectorizer.word_vocab), 
              out_vocab_size=len(vectorizer.word_vocab), 
              batch_first=True)

Loading vectorizer!


### quick inspection of model performance 

Before we load the model weights, what do the samples look like? 

In [14]:
# initial vectors
initial_hidden = make_initial_hidden(batch_size=8, hidden_size=args.zoo['parameters']['hidden_size'])
initial_x = make_initial_x(batch_size=8, vectorizer=vectorizer)

# sampled matrix of indices
sample_matrix = sample(net.emb, net.rnn, net.fc, 
                       initial_hidden, initial_x,
                       temp=0.8, n=5)

# decode matrix into text!
decode_matrix(vectorizer, sample_matrix)

['thru PACs Willie TAKE joint',
 'TACKY 52 855 Celebration Policy',
 'PENCE unauthorized trying copies burn',
 'Recruitment approximately CommonCore state deplorables',
 'am we arriving II guns',
 'Wayne throughly Thank Course prefer',
 'Mobile mr substance DOMINATING Drops',
 'falsify Republicanpresidentialdebate CBS gratifying !!']

###  loading model weights

In [15]:
if args.load_zoo_model and os.path.exists(args.zoo['filename']):
    print("Loading state dict!")
    net.load_state_dict(torch.load(args.zoo['filename'], map_location=lambda storage, loc: storage))
else:
    print("Using newly initiated network!")

Loading state dict!


### introspection post model weight load

notice we drop the `n` argument to sample this time. This was done assuming pretrained model weights were loaded.  Thish is because we assume the model is able to predict the end of sequence now! before, it could potentially go on forever. 

In [25]:
# initial vectors
initial_hidden = make_initial_hidden(batch_size=8, hidden_size=args.zoo['parameters']['hidden_size'])
initial_x = make_initial_x(batch_size=8, vectorizer=vectorizer)

# sampled matrix of indices
sample_matrix = sample(net.emb, net.rnn, net.fc, 
                       initial_hidden, initial_x,
                       temp=0.8)

# decode matrix into text!
decode_matrix(vectorizer, sample_matrix)

['Poll is going to say .. for Trump , but I have been saying ! # Utah " We',
 'I got to my office ALL . Info :',
 'Poll : Donald Trump " Trump works Hotel and women and focus for the incredible for Trump and the people',
 '" Clinton Dallas 1 / 15 - Breitbart & amp ; discuss :',
 'I hope you coming out of the race . Refuse ."',
 'Poll yet she cant even started to run for her misconduct in . Bad system is stagnant back !',
 'with you are on at 7 : 02 A . M . Enjoy !',
 'I know polls , who has win for the great police in the polls , I will be there .']

In [29]:
if args.cuda:
    print("CUDA mode enabled")
    net = net.cuda()
else:
    print("CUDA mode not enabled")
    net = net.cpu()
    
# optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=args.learning_rate)


# loss function

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

# progress bars

epoch_bar = tqdm_notebook(desc='epochs', total=args.num_epochs, position=1)

num_train_batches = len(train_dataset) // args.batch_size
train_bar = tqdm_notebook(desc='training', total=num_train_batches, position=2)

num_test_batches = len(test_dataset) // args.batch_size
test_bar = tqdm_notebook(desc='test', total=num_test_batches, position=3)

# history

train_loss_history = []
train_accuracy_history = []

test_loss_history = []
test_accuracy_history = []


try:
    for _ in range(args.num_epochs):
        batch_generator = generate_batches(train_dataset, batch_size=args.batch_size, use_cuda=args.cuda)
        
        per_epoch_loss = []
        per_epoch_accuracy = []
        
        net.train()
            
        for batch_dict in batch_generator:
            
            # step 1
            optimizer.zero_grad()
            
            # step 2
            y_pred = net(batch_dict['x_words'], batch_dict['x_lengths'])
            y_target = batch_dict['y_words']
            
            # step 3
            loss = sequence_loss(y_pred, y_target, IGNORE_INDEX_VALUE)
            
            # step 4
            loss.backward()
            optimizer.step()
          
            # bonus steps: bookkeeping
            
            per_epoch_loss.append(loss.cpu().data[0])
            
            accuracy = compute_accuracy(y_pred, batch_dict['y_words'], IGNORE_INDEX_VALUE)
            per_epoch_accuracy.append(accuracy)

            train_bar.update()
            
            train_bar.set_postfix(loss=per_epoch_loss[-1], 
                                  accuracy=per_epoch_accuracy[-1])
            
        train_loss_history.append(np.mean(per_epoch_loss))
        train_accuracy_history.append(np.mean(per_epoch_accuracy))
        
        # loop over test dataset
        
        batch_generator = generate_batches(test_dataset, batch_size=args.batch_size, use_cuda=args.cuda)
        
        per_epoch_loss = []
        per_epoch_accuracy = []
            
        # set it to eval mode; this turns stochastic functions off
        net.eval()
            
        for batch_dict in batch_generator:
            # step 1: compute output
            y_pred = net(batch_dict['x_words'], batch_dict['x_lengths'])
            y_target = batch_dict['y_words'] 
            
            # step 2: compute metrics
            loss = sequence_loss(y_pred, y_target, IGNORE_INDEX_VALUE)
            per_epoch_loss.append(loss.cpu().data[0])
          
            accuracy = compute_accuracy(y_pred, batch_dict['y_words'], IGNORE_INDEX_VALUE)
            per_epoch_accuracy.append(accuracy)

            test_bar.update()
            
            test_bar.set_postfix(loss=per_epoch_loss[-1], 
                                 accuracy=per_epoch_accuracy[-1])
            
        test_loss_history.append(np.mean(per_epoch_loss))
        test_accuracy_history.append(np.mean(per_epoch_accuracy))
        
        # update bars
        
        epoch_bar.set_postfix(train_loss=train_loss_history[-1], 
                              train_accuracy=train_accuracy_history[-1],
                              test_loss=test_loss_history[-1],
                              test_accuracy=test_accuracy_history[-1])
        epoch_bar.update()
        test_bar.n = 0
        train_bar.n = 0
        
except KeyboardInterrupt:
    print("...")

CUDA mode enabled


HBox(children=(IntProgress(value=0, description='epochs'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='training', max=38), HTML(value='')))

HBox(children=(IntProgress(value=0, description='test', max=9), HTML(value='')))

...


## Hypothetical Scenario

Up until now, you haven't had pretrained weights. You've only been using the freshly initialized network. Suddenly, you realize pre-trained word vectors exist and can be used in your network!

So, we will start by loading the glove word vectors and then incorporating them into our network

In [30]:
def load_word_vectors(filename):
    word_to_index = {}
    word_vectors = []
    
    with open(filename) as fp:
        for line in tqdm_notebook(fp.readlines(), leave=False):
            line = line.split(" ")
            
            word = line[0]
            word_to_index[word] = len(word_to_index)
            
            vec = np.array([float(x) for x in line[1:]])
            word_vectors.append(vec)
    word_vector_size = len(word_vectors[0])
    return word_to_index, word_vectors, word_vector_size

word_to_index, word_vectors, word_vector_size = load_word_vectors(args.glove_filename)

HBox(children=(IntProgress(value=0, max=400000), HTML(value='')))

now, we want to collate what we have from the word vectors with what is is on our vocabulary!

In [31]:
net.emb.weight.size() 

torch.Size([10311, 100])

In [32]:
net = WordRNN(embedding_size=zoo_params['embedding_size'], 
              hidden_size=zoo_params['hidden_size'],
              in_vocab_size=len(vectorizer.word_vocab), 
              out_vocab_size=len(vectorizer.word_vocab), 
              batch_first=True)
if args.cuda:
    print("CUDA mode enabled")
    net = net.cuda()
else:
    print("CUDA mode not enabled")
    net = net.cpu()

CUDA mode enabled


In [34]:
n = 0
for word, emb_index in tqdm_notebook(vectorizer.word_vocab.items(), leave=False):
    if word.lower() in word_to_index:
        n += 1
        glove_index = word_to_index[word.lower()]
        glove_vec = torch.FloatTensor(word_vectors[glove_index])
        if net.emb.weight.is_cuda:
            glove_vec = glove_vec.cuda()
        net.emb.weight.data[emb_index, :].set_(glove_vec)

print(n, 'replaced')

HBox(children=(IntProgress(value=0, max=10310), HTML(value='')))

9446 replaced


### Re-running training with these vectors

While you won't be able to really appreciate the gains if training on a cpu, if you were to run the code again (this is the same training routine as above), you would see faster convergence as a lot of the ground work is already done by the pre-trained embeddings!

In [35]:
if args.cuda:
    print("CUDA mode enabled")
    net = net.cuda()
else:
    print("CUDA mode not enabled")
    net = net.cpu()
    
# optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=args.learning_rate)


# loss function

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

# progress bars

epoch_bar = tqdm_notebook(desc='epochs', total=args.num_epochs, position=1)

num_train_batches = len(train_dataset) // args.batch_size
train_bar = tqdm_notebook(desc='training', total=num_train_batches, position=2)

num_test_batches = len(test_dataset) // args.batch_size
test_bar = tqdm_notebook(desc='test', total=num_test_batches, position=3)

# history

train_loss_history = []
train_accuracy_history = []

test_loss_history = []
test_accuracy_history = []


try:
    for _ in range(args.num_epochs):
        batch_generator = generate_batches(train_dataset, batch_size=args.batch_size, use_cuda=args.cuda)
        
        per_epoch_loss = []
        per_epoch_accuracy = []
        
        net.train()
            
        for batch_dict in batch_generator:
            
            # step 1
            optimizer.zero_grad()
            
            # step 2
            y_pred = net(batch_dict['x_words'], batch_dict['x_lengths'])
            y_target = batch_dict['y_words']
            
            # step 3
            loss = sequence_loss(y_pred, y_target, IGNORE_INDEX_VALUE)
            
            # step 4
            loss.backward()
            optimizer.step()
          
            # bonus steps: bookkeeping
            
            per_epoch_loss.append(loss.cpu().data[0])
            
            accuracy = compute_accuracy(y_pred, batch_dict['y_words'], IGNORE_INDEX_VALUE)
            per_epoch_accuracy.append(accuracy)

            train_bar.update()
            
            train_bar.set_postfix(loss=per_epoch_loss[-1], 
                                  accuracy=per_epoch_accuracy[-1])
            
        train_loss_history.append(np.mean(per_epoch_loss))
        train_accuracy_history.append(np.mean(per_epoch_accuracy))
        
        # loop over test dataset
        
        batch_generator = generate_batches(test_dataset, batch_size=args.batch_size, use_cuda=args.cuda)
        
        per_epoch_loss = []
        per_epoch_accuracy = []
            
        # set it to eval mode; this turns stochastic functions off
        net.eval()
            
        for batch_dict in batch_generator:
            # step 1: compute output
            y_pred = net(batch_dict['x_words'], batch_dict['x_lengths'])
            y_target = batch_dict['y_words'] 
            
            # step 2: compute metrics
            loss = sequence_loss(y_pred, y_target, IGNORE_INDEX_VALUE)
            per_epoch_loss.append(loss.cpu().data[0])
          
            accuracy = compute_accuracy(y_pred, batch_dict['y_words'], IGNORE_INDEX_VALUE)
            per_epoch_accuracy.append(accuracy)

            test_bar.update()
            
            test_bar.set_postfix(loss=per_epoch_loss[-1], 
                                 accuracy=per_epoch_accuracy[-1])
            
        test_loss_history.append(np.mean(per_epoch_loss))
        test_accuracy_history.append(np.mean(per_epoch_accuracy))
        
        # update bars
        
        epoch_bar.set_postfix(train_loss=train_loss_history[-1], 
                              train_accuracy=train_accuracy_history[-1],
                              test_loss=test_loss_history[-1],
                              test_accuracy=test_accuracy_history[-1])
        epoch_bar.update()
        test_bar.n = 0
        train_bar.n = 0
        
except KeyboardInterrupt:
    print("...")

CUDA mode enabled


HBox(children=(IntProgress(value=0, description='epochs'), HTML(value='')))

HBox(children=(IntProgress(value=0, description='training', max=38), HTML(value='')))

HBox(children=(IntProgress(value=0, description='test', max=9), HTML(value='')))

...


In [11]:
net = net.cpu()

# initial vectors
initial_hidden = make_initial_hidden(batch_size=8, hidden_size=args.zoo['parameters']['hidden_size'])
initial_x = make_initial_x(batch_size=8, vectorizer=vectorizer)

# sampled matrix of indices
sample_matrix = sample(net.emb, net.rnn, net.fc, 
                       initial_hidden, initial_x,
                       temp=0.8)

# decode matrix into text!
decode_matrix(vectorizer, sample_matrix)

['I support begging for ME , Donald , whod watch 12 .',
 'whose interview in Florida ! # Trump2016 # 2A " We Thanks law enforcement officers . Thank you for the',
 'who spoke about concerning it to run against " , Trump is not a news conference - dummy !',
 'who voted for one of the rigged system under budget that Secret Service never spoke to create it very well',
 'who voted for an absolutely ? is doing "',
 'Poll numbers was a great day . Focus severely place out of Illinois ! # TrumpPence16 # BigLeagueTruth # DemDebate',
 'I love very much . A real class is going on ?',
 'who voted for the presidency , and MSM is a great SILENT MAJORITY looming !',
 'I know it is 100 --- given !',
 'Poll , he just announced in USA ! Do make video plans to tell me to take the RNC and',
 'I will be interviewed by on at 8 : 30 A . M . So nice from Citi , 000',
 'I know that Trump gives his speech in history ? They burned us safe CNN - # VoteTrump today !"',
 'I know you are the worst New York ! Crooke